<a href="https://colab.research.google.com/github/rajshah4/makeMoE_simpsons/blob/main/makeMoE_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# makeMoE: Sparse Mixture of Experts from Scratch

## 1. Welcome and Learning Goals

This notebook expands on Andrej Karpathy's original makemore tutorial by layering in a sparse Mixture of Experts (MoE) architecture. The walkthrough is designed as a learning companion: we build every component from first principles, keep the code hackable, and surface prompts you can explore on your own.

**Learning goals**
- Understand how character-level language models ingest and encode raw text data.
- Reconstruct causal self-attention and see how it slots into a Transformer block.
- Implement sparse gating, experts, and routing to form a Mixture of Experts layer.
- Train the full model while tracking experiments and analysing generated samples.

**How to use this notebook**
1. Read the narrative in each section before running the code cells.
2. Experiment with the "Try it" prompts to internalise the ideas.
3. Keep notes on what changes improved or hurt performance—treat this notebook as a sandbox.


### Table of Contents
- [1. Welcome and Learning Goals](#1-welcome-and-learning-goals)
- [2. Project Setup](#2-project-setup)
- [3. Data Acquisition and Tokenization](#3-data-acquisition-and-tokenization)
- [4. Batching and Context Windows](#4-batching-and-context-windows)
- [5. Self-Attention Foundations](#5-self-attention-foundations)
- [6. Experts and Routing](#6-experts-and-routing)
- [7. Building the Sparse MoE Block](#7-building-the-sparse-moe-block)
- [8. Assembling the Language Model](#8-assembling-the-language-model)
- [9. Training and Experiment Tracking](#9-training-and-experiment-tracking)
- [10. Sampling and Evaluation](#10-sampling-and-evaluation)
- [11. Further Experiments and Learning Paths](#11-further-experiments-and-learning-paths)


![mixture of experts overview](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/moe.png)

## 2. Project Setup

This section makes sure your environment is ready to run the notebook smoothly. We install dependencies, import core libraries, and optionally configure experiment tracking.

> 💡 **Try it:** If you are working offline or prefer a different tracker, comment out the Weights & Biases cell and replace it with your tool of choice (Aim, Comet, TensorBoard, …).


In [ ]:
#Using Weights & Biases is entirely optional. I personally like to use W&B to track and log everything. You can pip install it easily.
%pip install wandb weave

### 2.1 Imports and Reproducibility

We rely almost entirely on PyTorch. Setting a manual seed keeps runs comparable when you tweak hyperparameters.

> 💡 **Try it:** Change the seed and see how early validation loss varies. Does model performance stabilize as you increase the number of training iterations?  What does that tell us about hte importance of the seed?


In [ ]:
#Import the necessary packages and set seed for reproducibility. For this notebook, pytorch is all you need
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)
#Optional
import wandb

### 2.2 Optional: Configure Experiment Tracking

Authenticate with Weights & Biases (or skip this cell if you prefer). For shared notebooks, set the API key through environment variables outside the notebook when possible.

> 💡 **Try it:** Replace the hardcoded key with `wandb.login()` or a secrets manager integration if you want to share this notebook safely.


In [ ]:
import os
os.environ['WANDB_API_KEY'] = ''

## 3. Data Acquisition and Tokenization

We will work with the Simpsons script dataset (a fun alternative to the original Shakespeare corpus). The next cells download the text, preview it, and build the integer encodings our model understands.

**Steps ahead**
1. Fetch the raw corpus.
2. Inspect a sample to understand style and vocabulary.
3. Build mappings between characters and integer ids.
4. Encode the entire dataset and split into train/validation sets.

> 💡 **Try it:** Swap in any other plain-text corpus. Update the download path and revisit the vocabulary size—how does pop lyrics or code differ from TV dialogue?  
Get shakespeare from here: #!wget https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/input.txt


In [ ]:
# read it in to inspect it
with open('simpsons.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, the rest validation (val)
train_data = data[:n]
val_data = data[n:]

## 4. Batching and Context Windows

Transformers consume fixed-length context windows. These cells explore how we construct batches of contiguous characters and targets for next-token prediction.

We are going to build the core training loop here so given the context (x), predict the next token (y).

> 💡 **Try it:** Experiment with larger `block_size` values to see how context length influences memory usage and validation loss. You may also try variable-length batching for a stretch goal.


In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
# setup next token prediction 
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

In [ ]:
# get some random starting positions
ix = torch.randint(len(data) - block_size, (batch_size,))
ix

In [ ]:
# Build input (x) and target (y)
x = torch.stack([data[i:i+block_size] for i in ix])
y = torch.stack([data[i+1:i+block_size+1] for i in ix])
x

In [ ]:
y

### 4.1 Visualising Autoregressive Behaviour

This printout shows how each target token is conditioned on an ever-growing context. Inspecting it helps demystify how character-level models learn next-step predictions.

> 💡 **Try it:** Modify `batch_size` or `block_size` and rerun the cell to see how the table changes. This is a quick way to debug your batching logic.


In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

## 5. Self-Attention Foundations

Before we add experts, we revisit causal self-attention. You will see how queries, keys, values, and masking work together to respect autoregressive ordering.

> 💡 **Try it:** Temporarily remove the causal mask (`tril`) and observe how the model cheats by attending to future tokens—validation loss will drop suspiciously fast.


![scaled dot product self attention](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/self_attention.png)

The provided code demonstrates self-attention's mechanics and fundamental concepts, specifically focusing on the classic scaled dot product self-attention. In this variant, the query, key, and value matrices all originate from the same input sequence. To ensure the integrity of the autoregressive language generation process, particularly in a decoder-only model, the code implements masking. This masking technique is crucial as it obscures any information following the current token's position, thereby directing the model's attention to only the preceding parts of the sequence. Such an attention mechanism is known as causal self-attention. It's important to note that the Sparse Mixture of Experts model isn't restricted to decoder-only Transformer architectures. In fact, much of the significant work in this field, particularly that by Shazeer et al, revolves around the T5 architecture, which encompasses both encoder and decoder components in the Transformer model.

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch size, sequence length (time steps), number of channels (embedding dimension)
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)

# wei:weights - attention scores (before softmax), shape (B, T, T)
# Each [b, t1, t2] entry is the dot product between the query at position t1 and the key at position t2 for batch b
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

# tril: lower-triangular matrix of shape (T, T), used to mask out future positions (causal masking)
tril = torch.tril(torch.ones(T, T))

# Mask out (set to -inf) all positions in wei where tril == 0 (i.e., future positions)
wei = wei.masked_fill(tril == 0, float('-inf'))

# Apply softmax to get normalized attention weights along the last dimension
wei = F.softmax(wei, dim=-1) # (B, T, T)

v = value(x) # (B, T, 16)
out = wei @ v # (B, T, T) @ (B, T, 16) -> (B, T, 16)
# The output from this final matrix product is subsequently passed through a linear layer as shown in the diagram above

out.shape

Visual diagram of what is happening
```
         ┌───────────────┐
 Input → │ Linear (Q,K,V)│
         └─────┬─┬─┬─────┘
               │ │ │
               Q K V
               │ │ │
            ┌──▼─┴─▼──┐
            │ Dot Prod │  → wei (B, T, T)
            └───┬─────┘
                │ (mask + softmax)
                ▼
          Attention Weights
                │
                ▼
         Weighted Sum with V
                │
                ▼
              Output (B, T, 16)

```


### 5.1 Modularising Attention Heads

We encapsulate the single-head computation inside a `Head` module. This mirrors the design in PyTorch's official Transformer implementation.

> 💡 **Try it:** Add dropout to the value projections or switch the activation in the attention weights to experiment with regularisation techniques.


In [ ]:
#Causal scaled dot product self-Attention Head
# This code block defines the Head class for a single self-attention head, along with some hyperparameters.
# This is based on the earlier attention head

n_embd = 64
n_head = 4
n_layer = 4
head_size = 16
dropout = 0.1

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [ ]:
#Multi-Headed Self Attention
# This class implements multi-head self-attention, a core component of the Transformer architecture.
# It runs several attention "heads" in parallel, each with its own set of parameters, and then projects their concatenated outputs back to the original embedding size.

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # Create a list of independent attention heads
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # Linear projection to combine the outputs of all heads
        self.proj = nn.Linear(n_embd, n_embd)
        # Dropout for regularization after projection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Run each head in parallel and concatenate their outputs along the last dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Project the concatenated outputs back to the embedding dimension and apply dropout
        out = self.dropout(self.proj(out))
        return out


In [ ]:
# This code checks whether the MultiHeadAttention module returns an output with the same embedding size as its input.
B, T, C = 4, 8, 64  # batch size, sequence length, embedding dimension
x = torch.randn(B, T, C)  # create a random input tensor
mha = MultiHeadAttention(num_heads=4, head_size=16)  # instantiate multi-head attention (4 heads × 16 = 64)
output = mha(x)
print(output.shape)  # should print: torch.Size([4, 8, 64])

## 6. Experts and Routing

Sparse MoE layers replace the dense feed-forward block with a pool of experts. Only a subset runs per token, which gives us capacity without proportional compute.

> 💡 **Try it:** Change the number of experts or their hidden size. How does capacity trade off with latency on your hardware?


In the Sparse Mixture of Experts (MoE) architecture, the self-attention mechanism within each transformer block remains unchanged. However, a notable alteration occurs in the structure of each block: the standard feed-forward neural network is replaced with several sparsely activated feed-forward networks, known as experts. "Sparse activation" refers to the process where each token in the sequence is routed to only a limited number of these experts – typically one or two – out of the total pool available. This modification allows for specialized processing of different parts of the input data, enabling the model to handle a wider range of complexities efficiently.

![experts](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/experts.png)

In [ ]:
# Expert module: defines a single expert as a small feed-forward neural network (MLP)
class Expert(nn.Module):
    """
    An Expert is a simple MLP (Multi-Layer Perceptron) consisting of:
    - A linear transformation to increase the embedding dimension (hidden layer)
    - A non-linearity (ReLU activation)
    - A linear transformation to project back to the original embedding dimension
    - Dropout for regularization
    """

    def __init__(self, n_embd):
        super().__init__()
        # The expert's network: input -> hidden (4x size) -> ReLU -> output -> Dropout
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Expands embedding dimension by 4x
            nn.ReLU(),                      # Non-linear activation
            nn.Linear(4 * n_embd, n_embd),  # Projects back to original embedding size
            nn.Dropout(dropout),             # Dropout for regularization
        )

    def forward(self, x):
        # Pass input through the expert's MLP
        return self.net(x)

### 6.1 Top-k Gating Intuition

The router scores each expert before selecting the top-k candidates per token. Keeping the walkthrough example grounded in small tensors makes the later implementation easier to follow.

> 💡 **Try it:** Visualise the gating logits with a heatmap to check whether certain experts dominate. You can also try sampling experts proportionally rather than taking the strict top-k.


![top k gating](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/topk.png)

The gating network, also known as the router, determines which expert network receives the output for each token from the multi-head attention. Let's consider a simple example: suppose there are 4 experts, and the token is to be routed to the top 2 experts. Initially, we input the token into the gating network through a linear layer. This layer projects the input tensor from a shape of (2, 4, 32) — representing (Batch size, Tokens, n_embed, where n_embed is the channel dimension of the input) — to a new shape of (2, 4, 4), which corresponds to (Batch size, Tokens, num_experts), where num_experts is the count of expert networks. Following this, we determine the top k=2 highest values and their respective indices along the last dimension.

In [ ]:
# ----------------------------------------
# Example: Understanding how top-k gating works in a Mixture of Experts (MoE) model

# Number of experts in the MoE layer
num_experts = 4

# Number of experts to select (top-k) for each token
top_k = 2

# Embedding dimension of each token
n_embed = 32

# Simulate the output from a multi-head attention block
# Shape: (batch_size=2, context_length=4, n_embed=32)
mh_output = torch.randn(2, 4, n_embed)

# Linear layer to produce logits for each expert from the token embeddings
# Projects from n_embed to num_experts for each token
topkgate_linear = nn.Linear(n_embed, num_experts)  # nn.Linear(32, 4)

# Compute the logits for each expert for every token
logits = topkgate_linear(mh_output)

# Select the top-k expert logits and their indices for each token
# top_k_logits: the values of the top-k logits
# top_k_indices: the indices of the top-k experts
top_k_logits, top_k_indices = logits.topk(top_k, dim=-1)  # Get top-k experts

# Display the top-k logits and their corresponding expert indices
top_k_logits, top_k_indices

Obtain the sparse gating output by only keeping the top k values in their respective index along the last dimension. Fill the rest with '-inf' and pass through a softmax activation. This pushes '-inf' values to zero, makes the top two values more accentuated and sum to 1. This summation to 1 helps with the weighting of expert outputs

In [ ]:
# Create a tensor of the same shape as logits, filled with -inf.
# This will be used to mask out all but the top-k expert logits for each token.
zeros = torch.full_like(logits, float('-inf'))  # full_like clones the shape and dtype of logits, filling with -inf.

# Use scatter to place the top-k logits at their corresponding expert indices.
# All other positions remain -inf, so only the top-k experts are "active" for each token.
sparse_logits = zeros.scatter(-1, top_k_indices, top_k_logits)

# Display the sparse logits tensor, where only the top-k expert positions have values, others are -inf.
sparse_logits

In [ ]:
# Apply softmax to the sparse logits along the last dimension to obtain the gating output.
# This converts the -inf-masked logits into a sparse probability distribution,
# where only the top-k expert positions have nonzero values (sum to 1 for each token).
gating_output = F.softmax(sparse_logits, dim=-1)
gating_output  # Display the resulting sparse gating output tensor

### 6.2 Noisy Top-k Routing

Adding stochasticity to the router encourages load balancing: so different experts get a chance to learn before the model during training.

> 💡 **Try it:** Tune the noise scale or swap in Gumbel noise. Monitor the router entropy during training to ensure the load is balanced.


In [ ]:
# Define the Top-k Router module for selecting top-k experts per token
class TopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        """
        Args:
            n_embed (int): Dimensionality of input embeddings.
            num_experts (int): Number of experts in the MoE layer.
            top_k (int): Number of top experts to route each token to.
        """
        super(TopkRouter, self).__init__()
        self.top_k = top_k
        # Linear layer to produce routing logits for each expert
        self.linear = nn.Linear(n_embed, num_experts)

    def forward(self, mh_output):
        """
        Args:
            mh_output (Tensor): Output tensor from the multi-head self-attention block,
                                shape (..., n_embed).
        Returns:
            router_output (Tensor): Sparse softmax probabilities over experts,
                                    only top-k per token are nonzero.
            indices (Tensor): Indices of the top-k experts for each token.
        """
        # Compute routing logits for each expert
        logits = self.linear(mh_output)
        # Select the top-k logits and their indices along the expert dimension
        top_k_logits, indices = logits.topk(self.top_k, dim=-1)
        # Create a tensor filled with -inf to mask out non-top-k experts
        zeros = torch.full_like(logits, float('-inf'))
        # Scatter the top-k logits into their respective positions; others remain -inf
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        # Apply softmax to obtain sparse gating probabilities (only top-k nonzero)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices


In [ ]:
#Testing this out:
num_experts = 4
top_k = 2
n_embd = 32

mh_output = torch.randn(2, 4, n_embd)  # Example input
top_k_gate = TopkRouter(n_embd, num_experts, top_k)
gating_output, indices = top_k_gate(mh_output)
gating_output.shape, gating_output, indices
#And it works!!

Althought the mixtral paper released recently does not make any mention of it, I believe Noisy top-k Gating is an important tool in training MoE models. Essentially, you don't want all the tokens to be sent to the same set of 'favored' experts. You want a fine balance of exploitation and exploration. For this purpose, to load balance, it is helpful to add standard normal noise to the logits from the gating linear layer. This makes training more efficient

![noisy top-k gating](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/noisytopkgating.png)

In [ ]:
#Changing the above to accomodate noisy top-k gating
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        #layer for router logits
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear =nn.Linear(n_embed, num_experts)


    def forward(self, mh_output):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.topkroute_linear(mh_output)

        #Noise logits
        noise_logits = self.noise_linear(mh_output)

        #Adding scaled unit gaussian noise to the logits
        noise = torch.randn_like(logits)*F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices

In [ ]:
#Testing this out, again:
num_experts = 8
top_k = 2
n_embd = 16

mh_output = torch.randn(2, 4, n_embd)  # Example input
noisy_top_k_gate = NoisyTopkRouter(n_embd, num_experts, top_k)
gating_output, indices = noisy_top_k_gate(mh_output)
gating_output.shape, gating_output, indices
#It works!!

## 7. Building the Sparse MoE Block

Now we bring routing and experts together. The `SparseMoE` module applies experts selectively and aggregates their outputs with gating weights.

> 💡 **Try it:** Profile how many tokens each expert processes per batch. You can log the routing decisions to W&B for diagnostics.


The primary aspect of this process involves the gating network's output. After acquiring these results, the top k values are selectively multiplied with the outputs from the corresponding top-k experts for a given token. This selective multiplication forms a weighted sum, which constitutes the SparseMoe block's output. The critical and challenging part of this process is to avoid unnecessary multiplications. It's essential to conduct forward passes only for the top_k experts and then compute this weighted sum. Performing forward passes for each expert would defeat the purpose of employing a sparse MoE, as it would no longer be sparse.

In [ ]:
class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(SparseMoE, self).__init__()
        # Router determines which experts to use for each token
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        # List of expert networks (each is an MLP)
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        # x: input tensor of shape (batch, seq_len, n_embed)
        # Get gating weights and top-k expert indices from the router
        gating_output, indices = self.router(x)
        # Initialize output tensor (same shape as input)
        final_output = torch.zeros_like(x)

        # Flatten input and gating output for easier indexing
        flat_x = x.view(-1, x.size(-1))  # (batch*seq_len, n_embed)
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))  # (batch*seq_len, num_experts)

        # Iterate over each expert
        for i, expert in enumerate(self.experts):
            # Create a mask for tokens where this expert is in the top-k
            expert_mask = (indices == i).any(dim=-1)  # (batch, seq_len)
            flat_mask = expert_mask.view(-1)  # (batch*seq_len,)

            # Only process tokens assigned to this expert
            if flat_mask.any():
                # Select inputs for this expert
                expert_input = flat_x[flat_mask]  # (num_selected, n_embed)
                # Forward pass through the expert
                expert_output = expert(expert_input)  # (num_selected, n_embed)

                # Get the corresponding gating scores for this expert
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)  # (num_selected, 1)
                # Weight the expert outputs by the gating scores
                weighted_output = expert_output * gating_scores  # (num_selected, n_embed)

                # Add the weighted outputs to the final output tensor
                # (scatter-add for all tokens assigned to this expert)
                final_output[expert_mask] += weighted_output.squeeze(1)

        # Return the aggregated output from all experts
        return final_output

In [ ]:
import torch
import torch.nn as nn

# --- Demo: Testing SparseMoE Routing and Aggregation ---

# Parameters for the test
num_experts = 8
top_k = 2
n_embd = 16
dropout = 0.1
batch_size = 4
seq_len = 8

# Create a random input tensor simulating the output of a multi-head attention block
mh_output = torch.randn(batch_size, seq_len, n_embd)

# Instantiate the SparseMoE module
sparse_moe = SparseMoE(n_embd, num_experts, top_k)

# Forward pass through the SparseMoE
final_output = sparse_moe(mh_output)

print("Input shape (batch, seq_len, n_embd):", mh_output.shape)
print("Output shape (should match input):", final_output.shape)

# This prints the sum of all values in the final output tensor from the SparseMoE layer.
# If the routing and aggregation are working, this sum should be nonzero (since the experts are producing outputs and the gating is nontrivial).
# A zero sum would indicate that either all gating weights are zero or the experts are not contributing, which would be a bug.
print("Sum of output tensor (should be nonzero):", final_output.sum().item())

# --- Deeper Test: Check that only top-k experts are active per token ---

# To do this, we can directly use the router to inspect the gating and indices
with torch.no_grad():
    gating_weights, topk_indices = sparse_moe.router(mh_output)
    # gating_weights: (batch, seq_len, num_experts)
    # topk_indices: (batch, seq_len, top_k)
    print("\nSample of top-k expert indices for first batch element:")
    print(topk_indices[0])  # shape: (seq_len, top_k)
    print("Gating weights for first token (should sum to <= 1):")
    print(gating_weights[0, 0])
    print("Sum of gating weights for first token:", gating_weights[0, 0].sum().item())

# --- Sanity check: If all experts are used (top_k = num_experts), output should be a dense mixture ---
dense_moe = SparseMoE(n_embd, num_experts, num_experts)
dense_output = dense_moe(mh_output)
print("\nDense MoE output shape:", dense_output.shape)

# This line computes the L2 norm (Euclidean distance) between the outputs of the sparse MoE (which only routes each token to its top-k experts)
# and the dense MoE (which routes each token to all experts, i.e., top_k = num_experts).
# A nonzero difference shows that the sparse routing is actually selecting a subset of experts, not just acting like a dense mixture.
print("Difference between sparse and dense output (L2 norm):", (final_output - dense_output).norm().item())

### 7.1 Weighted Expert Aggregation

Remember that the gating weights are just as important as the expert outputs—they determine how much each expert contributes to the final representation.

> 💡 **Try it:** Try alternative combination strategies such as residual mixing or learnable temperature on the softmax.


![sparse MoE](https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/images/sparseMoEfinal.png)

## 8. Assembling the Language Model

With the sparse MoE block ready, we can stack Transformer blocks, add embeddings, and define the language model head.

> 💡 **Try it:** Compare this sparse architecture with a dense baseline by swapping the MoE block for a standard MLP. Track parameters vs. performance.


In [ ]:
#First defining hyperparameters and boiler plate code. Imports and data preparation code is repeated for convenience
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import init

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 400
head_size = 16
n_embed = 128
n_head = 8
n_layer = 8
dropout = 0.1
num_experts = 8
top_k = 2
# ------------

torch.manual_seed(1337)

with open('simpsons.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

#Multi-Headed Self Attention
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [ ]:
#Expert module
class Expert(nn.Module):
    """ An MLP is a simple linear layer followed by a non-linearity i.e. each Expert """

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

#noisy top-k gating
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        #layer for router logits
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear =nn.Linear(n_embed, num_experts)


    def forward(self, mh_output):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.topkroute_linear(mh_output)

        #Noise logits
        noise_logits = self.noise_linear(mh_output)

        #Adding scaled unit gaussian noise to the logits
        noise = torch.randn_like(logits)*F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices

#Now create the sparse mixture of experts module
class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(SparseMoE, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)

        # Reshape inputs for batch processing
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        # Process each expert in parallel
        for i, expert in enumerate(self.experts):
            # Create a mask for the inputs where the current expert is in top-k
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)

                # Extract and apply gating scores
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores

                # Update final output additively by indexing and adding
                final_output[expert_mask] += weighted_output.squeeze(1)

        return final_output

In [ ]:
#First create a self attention + mixture of experts block, that may be repeated several number of times
#Copy pasting key architecture variables for clarity

class Block(nn.Module):
    """ Mixture of Experts Transformer block: communication followed by computation (multi-head self attention + SparseMoE) """

    def __init__(self, n_embed, n_head, num_experts, top_k):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.smoe = SparseMoE(n_embed, num_experts, top_k)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.smoe(self.ln2(x))
        return x

In [ ]:
#Finally putting it all together to crease a sparse mixture of experts language model
class SparseMoELanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head, num_experts=num_experts,top_k=top_k) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

### 8.1 Weight Initialisation Notes

We use Kaiming (He) initialization by default for the weights in our expert networks because these networks use ReLU activations. Kaiming initialization is specifically designed to maintain stable signal variance through layers with ReLU, helping gradients flow well and preventing vanishing/exploding issues during training.
 
If you use a different activation function (such as Tanh or GELU), you might want to use a different initialization scheme, like Xavier (Glorot) initialization, which is better suited for activations that are symmetric around zero. The choice of initialization can have a significant impact on training stability and convergence speed, so it's a good idea to match your initialization to your activation function.
 
You can experiment with per-layer or per-module initialization strategies. For example, you might want to use a smaller variance for the router's weights to help stabilize routing decisions early in training, while keeping Kaiming initialization for the expert MLPs.

> 💡 **Try it:** Implement per-layer initialisation schemes—e.g. try smaller variance for the router to stabilise early training.


In [ ]:

def kaiming_init_weights(m):
    if isinstance (m, (nn.Linear)):
        init.kaiming_normal_(m.weight)

In [ ]:
model = SparseMoELanguageModel()
model.apply(kaiming_init_weights)

## 9. Training and Experiment Tracking

Training loops are more fun when you can compare runs. This section logs hyperparameters, losses, checkpoints, and generated samples to Weights & Biases (with a graceful fallback if you prefer to stay offline).

> 💡 **Try it:** Capture additional metrics—router entropy, number of tokens per expert, gradient norms—to deepen your understanding of training dynamics.


In [ ]:
#Using Weights & Biases
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Define generation checkpoints
generation_steps = [0,100,500,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]

# Simple local logging when W&B is not available
local_logs = []

# Function to generate text samples
def generate_samples(model, step, num_samples=3, max_tokens=200):
    """Generate text samples and save them"""
    model.eval()
    samples = []
    
    with torch.no_grad():
        for i in range(num_samples):
            # Start with a random character from the vocabulary
            context = torch.randint(0, vocab_size, (1, 1), device=device)
            generated = model.generate(context, max_new_tokens=max_tokens)
            text = decode(generated[0].tolist())
            samples.append(text)
    
    model.train()
    return samples

# Function to save model checkpoint
def save_model_checkpoint(model, step, optimizer):
    """Save model checkpoint as W&B artifact"""
    checkpoint_path = f"model_checkpoint_step_{step}.pt"
    torch.save({
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    
    # Create W&B artifact if W&B is available
    if use_wandb:
        try:
            artifact = wandb.Artifact(f"model_checkpoint_step_{step}", type="model")
            artifact.add_file(checkpoint_path)
            wandb.log_artifact(artifact)
            print(f"✅ Model checkpoint uploaded to W&B: {checkpoint_path}")
        except Exception as e:
            print(f"⚠️  W&B artifact upload failed: {e}")
            print(f"✅ Model checkpoint saved locally: {checkpoint_path}")
    else:
        print(f"✅ Model checkpoint saved locally: {checkpoint_path}")
    
    return checkpoint_path

# Initialize W&B with error handling
try:
    wandb.init(
        project="makeMoE",
        config={
            "batch_size": batch_size,
            "block_size": block_size,
            "max_iters": max_iters,
            "eval_interval": eval_interval,
            "learning_rate": learning_rate,
            "device": device,
            "eval_iters": eval_iters,
            "dropout": dropout,
            "num_experts": num_experts,
            "top_k": top_k,
            "generation_steps": generation_steps
        },
        # Disable automatic code logging to avoid connection issues
        settings=wandb.Settings(
            _disable_stats=True,
            _disable_meta=True,
            code_dir=None
        )
    )
    print("✅ W&B initialized successfully!")
    use_wandb = True
except Exception as e:
    print(f"⚠️  W&B initialization failed: {e}")
    print("Continuing without W&B logging...")
    use_wandb = False

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        # Log metrics to W&B
        if use_wandb:
            try:
                wandb.log({
                    "train_loss": float(losses['train']),
                    "val_loss": float(losses['val']),
                    "step": iter
                })
            except Exception as e:
                print(f"⚠️  W&B logging failed: {e}")
        else:
            # Local logging fallback
            local_logs.append({
                "step": iter,
                "train_loss": float(losses['train']),
                "val_loss": float(losses['val'])
            })

    # Generate samples and save model at specified steps
    if iter in generation_steps:
        print(f"\n=== Generating samples at step {iter} ===")
        samples = generate_samples(model, iter)
        
        # Log samples to W&B
        for i, sample in enumerate(samples):
            print(f"Sample {i+1}:")
            print(sample[:500] + "..." if len(sample) > 500 else sample)
            print("-" * 50)
            
            # Log to W&B as text
            if use_wandb:
                try:
                    wandb.log({
                        f"generation_step_{iter}_sample_{i+1}": wandb.Html(f"<pre>{sample}</pre>"),
                        "step": iter
                    })
                except Exception as e:
                    print(f"⚠️  W&B generation logging failed: {e}")
            else:
                # Local logging fallback
                local_logs.append({
                    "type": "generation",
                    "step": iter,
                    "sample": i+1,
                    "text": sample
                })
        
        # Save model checkpoint
        checkpoint_path = save_model_checkpoint(model, iter, optimizer)
        print(f"Model checkpoint saved: {checkpoint_path}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Save final model
print("\n=== Saving final model ===")
final_samples = generate_samples(model, max_iters, num_samples=5, max_tokens=300)
for i, sample in enumerate(final_samples):
    print(f"Final Sample {i+1}:")
    print(sample[:500] + "..." if len(sample) > 500 else sample)
    print("-" * 50)
    
    if use_wandb:
        try:
            wandb.log({
                f"final_generation_sample_{i+1}": wandb.Html(f"<pre>{sample}</pre>"),
                "step": max_iters
            })
        except Exception as e:
            print(f"⚠️  W&B final generation logging failed: {e}")

# Save final model checkpoint
final_checkpoint = save_model_checkpoint(model, max_iters, optimizer)
print(f"Final model checkpoint saved: {final_checkpoint}")

# Save local logs if W&B was not available
if not use_wandb and local_logs:
    import json
    with open("training_logs.json", "w") as f:
        json.dump(local_logs, f, indent=2)
    print("✅ Local training logs saved to training_logs.json")

# Finish the W&B run
if use_wandb:
    try:
        wandb.finish()
        print("✅ W&B run finished successfully!")
    except Exception as e:
        print(f"⚠️  W&B finish failed: {e}")
else:
    print("ℹ️  Training completed without W&B logging")

In [ ]:
# Optional: Create a dataset of all generations for easy access
import json
from datetime import datetime

def create_generations_dataset():
    """Create a dataset containing all generations from training"""
    
    # This would collect all the generations that were logged during training
    # In a real scenario, you might want to store these in a more structured way
    
    generations_data = {
        "model_info": {
            "architecture": "SparseMoE",
            "num_experts": num_experts,
            "top_k": top_k,
            "vocab_size": vocab_size,
            "block_size": block_size,
            "n_embed": n_embed,
            "n_head": n_head,
            "n_layer": n_layer
        },
        "training_info": {
            "max_iters": max_iters,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "generation_steps": generation_steps
        },
        "generations": {
            # This would be populated with actual generations from training
            # For now, it's a placeholder structure
            "step_0": "Generated at step 0 - random initialization",
            "step_100": "Generated at step 100 - early training",
            "step_500": "Generated at step 500 - learning patterns",
            "step_1000": "Generated at step 1000 - improving coherence",
            "step_2000": "Generated at step 2000 - better structure",
            "step_3000": "Generated at step 3000 - final training",
            "step_4000": "Generated at step 4000 - better structure",
            "step_5000": "Generated at step 5000 - better structure",
            "step_6000": "Generated at step 6000 - better structure",
            "step_7000": "Generated at step 7000 - better structure",
            "step_8000": "Generated at step 8000 - better structure",
            "step_9000": "Generated at step 9000 - better structure",
            "step_10000": "Generated at step 10000 - better structure",
            "final": "Final generations after full training"
        },
        "created_at": datetime.now().isoformat()
    }
    
    # Save as JSON file
    with open("generations_dataset.json", "w") as f:
        json.dump(generations_data, f, indent=2)
    
    # Also save as W&B artifact if W&B is available
    try:
        if 'use_wandb' in globals() and use_wandb:
            artifact = wandb.Artifact("generations_dataset", type="dataset")
            artifact.add_file("generations_dataset.json")
            wandb.log_artifact(artifact)
            print("✅ Dataset uploaded to W&B!")
        else:
            print("ℹ️  Dataset saved locally (W&B not available)")
    except Exception as e:
        print(f"⚠️  W&B dataset upload failed: {e}")
        print("✅ Dataset saved locally")
    
    print("Generations dataset created and saved!")
    return generations_data

# Uncomment the line below to create the dataset
# generations_dataset = create_generations_dataset()

## 10. Sampling and Evaluation

Periodic sampling shows how the model's voice evolves. We capture generations at multiple training steps, archive checkpoints, and create a lightweight dataset of outputs.

> 💡 **Try it:** Add qualitative metrics—perplexity on held-out text, distinct-ngrams, or sentiment scores if you work with other corpora.


### 10.1 Monitoring Loss Curves

Validation loss is your early warning signal. Watch for divergence between train and validation to decide when to stop training or adjust regularisation.

> 💡 **Try it:** Plot additional diagnostics such as learning-rate warmup, or compare multiple runs within the W&B UI.


In [ ]:
# generate from the model. Not great. Not too bad either
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

## 11. Further Experiments and Learning Paths

Ready to explore further? Here are some directions:
- **Routing research:** Implement auxiliary load-balancing losses from the Switch Transformer or Mixtral papers.
- **Curriculum learning:** Start with shorter sequences before ramping up context length.
- **Tokenizer swaps:** Replace the character tokenizer with Byte Pair Encoding (BPE) to model longer-range structure.
- **Inference tricks:** Experiment with nucleus sampling, temperature schedules, or expert pruning for faster inference.
- **Visualisation:** Log attention maps and expert selections to better understand what each expert specialises in.

If you make interesting discoveries, consider packaging them as pull requests or blog posts—teaching others is the fastest way to solidify your understanding.
